In [1]:
import datasets
import os
import numpy as np
import sys
sys.path.append("../retrievers")

from tqdm import tqdm
from e2e_retriever import E2ERetriever

os.environ["HF_HOME"] = "/projects/ogma2/users/andrewsi/cache/huggingface/"
root = "/projects/ogma2/users/andrewsi/control-data2text"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
train_path = f"{root}/prototype-retrieval/datasets/e2e/e2e_train_v2"
retriever = E2ERetriever(train_path)

In [3]:
retriever.dataset

Dataset({
    features: ['edit_dist_map', 'edit_dist_map_no_mask', 'masked_target', 'masked_target_embed', 'source', 'source_embed', 'split_masked_target', 'split_target', 'target', 'target_embed', 'values'],
    num_rows: 29487
})

In [7]:
retriever.dataset = retriever.dataset.map(lambda example: {'split_target': example['target'].split()})

In [8]:
retriever.save_dataset()

In [4]:
retriever.dataset[0]['source']

'name The_Vaults eatType pub priceRange more_than_£30 customerRating 5_out_of_5 near Café_Adriatic'

In [6]:
retriever.add_embeds("source", gpu=0)

In [4]:
retriever.write_train_set(f"{root}/transformers/examples/seq2seq/test_data/e2e_proto/train_k3_no_mask.json", retrieval_map="edit_dist_map_no_mask", retrieval_embed=None, retrieval_k=3)

100%|██████████| 29487/29487 [05:46<00:00, 85.09it/s]
Wrote training data to file: /projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/e2e_proto/train_k3_no_mask.json


In [5]:
retrieval_k = 5
weighted = False
max_edit_dist = 10
retriever.write_train_set(f"/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/e2e_proto/train_k{retrieval_k}{'_max' + str(max_edit_dist) if max_edit_dist else ''}{'_weighted' if weighted else ''}.json", retrieval_k=retrieval_k, max_edit_dist=max_edit_dist, weighted=weighted)

100%|██████████| 29487/29487 [01:03<00:00, 464.58it/s]
Wrote training data to file: /projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/e2e_proto/train_k5_max10.json


In [16]:
train_dataset = datasets.load_from_disk("../datasets/e2e/e2e_train")

In [17]:
train_dataset

Dataset({
    features: ['edit_dist_map', 'masked_target', 'source', 'split_masked_target', 'target', 'values'],
    num_rows: 29487
})

In [18]:
train_dataset = train_dataset.map(lambda x: x, remove_columns=['edit_dist_map'])

In [20]:
train_dataset

Dataset({
    features: ['masked_target', 'source', 'split_masked_target', 'target', 'values'],
    num_rows: 29487
})

In [21]:
train_dataset.save_to_disk("../datasets/e2e/e2e_train_v2")